Мышковец С.А., v.1 1.05.2023

Решение задачи:

Сравнить существующие решения по саммаризации тестов.

Вывод:

При оценке саммаризации  сравниваются предикты с кратким содержанием, созданным человеком. Не совсем показательный критерий.
Модель facebook/bart-large-cnn обучена на датасете cnn daily и хорошо справляется с саммаризацией новостей.
Модель philschmid/bart-large-cnn-samsum фактически та же модель facebook/bart-large-cnn дотренированная  на датасете samsum и хорошо справляется с саммаризацией диалогов.
Обе модели делают extractive summarazation.


In [47]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 

import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

import evaluate

import seaborn as sns 
plt.style.use('ggplot')

import gc

# Metric Card for ROUGE

**Metric Description**
ROUGE, or Recall-Oriented Understudy for Gisting Evaluation, is a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing. The metrics compare an automatically produced summary or translation against a reference or a set of references (human-produced) summary or translation.

Note that ROUGE is case insensitive, meaning that upper case letters are treated the same way as lower case letters.

This metrics is a wrapper around the Google Research reimplementation of ROUGE

https://huggingface.co/spaces/evaluate-metric/rouge

In [ ]:
# pip install evaluate
# !pip install rouge-score

In [49]:
# Пример использования метрики ROUGE
rouge = evaluate.load('rouge')
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = rouge.compute(predictions=predictions,
                         references=references,
                            tokenizer=lambda x: x.split())
print(results)
# {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}


---

---

# MOST LIKES & MOST DOWNLOADS

# facebook/bart-large-cnn

**Model description**
BART is a transformer encoder-encoder (seq2seq) model with a bidirectional (BERT-like) encoder and an autoregressive (GPT-like) decoder. BART is pre-trained by (1) corrupting text with an arbitrary noising function, and (2) learning a model to reconstruct the original text.

BART is particularly effective when fine-tuned for text generation (e.g. summarization, translation) but also works well for comprehension tasks (e.g. text classification, question answering). This particular checkpoint has been fine-tuned on CNN Daily Mail, a large collection of text-summary pairs.

**Intended uses & limitations**
You can use this model for text summarization.

**Evaluation results**

ROUGE-1 on cnn_dailymail self-reported 42.949

ROUGE-2 on cnn_dailymail self-reported 20.815

ROUGE-L on cnn_dailymail self-reported 30.619

ROUGE-LSUM on cnn_dailymail self-reported 40.038 

loss on cnn_dailymail self-reported 2.529

gen_len on cnn_dailymail self-reported 78.587


Предположительно loss:

**criterion label_smoothed_cross_entropy**

https://github.com/facebookresearch/fairseq/blob/main/examples/bart/README.summarization.md#4-fine-tuning-on-cnn-dm-summarization-task
4) Fine-tuning on CNN-DM summarization task:
Example fine-tuning CNN-DM

TOTAL_NUM_UPDATES=20000  
WARMUP_UPDATES=500      
LR=3e-05
MAX_TOKENS=2048
UPDATE_FREQ=4
BART_PATH=/path/to/bart/model.pt

CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 fairseq-train cnn_dm-bin \
    --restore-file $BART_PATH \
    --max-tokens $MAX_TOKENS \
    --task translation \
    --source-lang source --target-lang target \
    --truncate-source \
    --layernorm-embedding \
    --share-all-embeddings \
    --share-decoder-input-output-embed \
    --reset-optimizer --reset-dataloader --reset-meters \
    --required-batch-size-multiple 1 \
    --arch bart_large \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.1 \
    --dropout 0.1 --attention-dropout 0.1 \
    --weight-decay 0.01 --optimizer adam --adam-betas "(0.9, 0.999)" --adam-eps 1e-08 \
    --clip-norm 0.1 \
    --lr-scheduler polynomial_decay --lr $LR --total-num-update $TOTAL_NUM_UPDATES --warmup-updates $WARMUP_UPDATES \
    --fp16 --update-freq $UPDATE_FREQ \
    --skip-invalid-size-inputs-valid-test \
    --find-unused-parameters;

In [ ]:
# pip install transformers

In [75]:
from transformers import pipeline

summarizer_facebook = pipeline("summarization", model="facebook/bart-large-cnn")

In [80]:
ARTICLE_from_docs = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

In [81]:
print(summarizer_facebook(ARTICLE_from_docs, max_length=130, min_length=30, do_sample=False))

[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


In [90]:
ARTICLE_bbc_news = """Residents of a tiny Swiss village have all been evacuated because of the risk of an imminent rockslide.

Brienz's fewer than 100 villagers were given just 48 hours to pack what they could and abandon their homes.

Even the dairy cows were loaded up for departure after geologists warned a rockfall was imminent.

Two million cubic metres of rock is coming loose from the mountain above, and a rockslide could obliterate the village.

The development has raised questions about the safety of some mountain communities, as global warming changes the alpine environment.

Brienz, in the eastern canton of Graubünden, is now empty.

The village has been judged a geological risk for some time and is built on land that is subsiding down towards the valley, causing the church spire to lean and large cracks to appear in buildings.

As the minutes ticked towards the deadline to leave, even Brienz's dairy cows were being taken to safety.

The residents, some young, some old, families, farmers and professional couples, had two days to abandon their homes.

They were asked earlier this week to evacuate the village by Friday evening.

Switzerland's Alpine regions are especially sensitive to global warming - as the permafrost high in the mountains begins to thaw, the rock becomes more unstable.

This particular mountain has always been unstable, but recently the rock has been shifting faster and faster.

Days of heavy rain could bring two million cubic metres of loosened rock crashing down the mountainside onto the village, scientists warned.

Now the villagers must wait, in temporary accommodation, for the rock to fall - and hope it misses their homes.
"""

In [97]:
human_produced_summary_bbc_news = """Residents of a tiny Swiss village have all been evacuated  within 48 hours due to the risk of an imminent rockslide. 
The residents must wait and hope their homes will not be destroyed by loosened rock crashing down the mountainside. 
The safety of mountain communities has been judged a geological risk for some time and is considered to be a global warming consequence."""

In [162]:
summarizer_facebook(ARTICLE_bbc_news, max_length=70, min_length=30, do_sample=False)

[{'summary_text': 'Residents of Brienz, in the eastern canton of Graubünden, were given just 48 hours to leave their homes. Two million cubic metres of rock is coming loose from the mountain above, and a rockslide could obliterate the village.'}]

In [124]:
# def prediction(summarizer):
#     for i in summarizer():
#      for k, v in i.items():
#         return v

In [ ]:
# prediction(summarizer_facebook(ARTICLE_bbc_news))

In [161]:
rouge = evaluate.load('rouge')
predictions = ["The village of Brienz in Graubünden is built on land that is subsiding down towards the valley. Two million cubic metres of rock is coming loose from the mountain above and could obliterate the village."]
# references = ["Residents of a tiny Swiss village have all been evacuated  within 48 hours due to the risk of an imminent rockslide. The residents must wait and hope their homes will not be destroyed by loosened rock crashing down the mountainside. The safety of mountain communities has been judged a geological risk for some time and is considered to be a global warming consequence."]
references = [human_produced_summary_bbc_news]
results = rouge.compute(predictions=predictions,
                        references=references,
                        # tokenizer=lambda x: x.split()
                            )
print(results)

{'rouge1': 0.24242424242424246, 'rouge2': 0.0, 'rougeL': 0.14141414141414138, 'rougeLsum': 0.16161616161616163}


In [ ]:
# Load a pretrained model - ValueError: No model config found in the file at .
# Saved weights, no model architecture

model = tf.keras.models.load_model('tf_model.h5')

In [133]:
ARTICLE_review= """I feel I have to write to keep others from wasting their money. 
This book seems to have been written by a 7th grader with poor grammatical skills for her age! 
As another reviewer points out, there is a misspelling on the cover, and I believe there is at least one per chapter. 
For example, it was mentioned twice that she had a "lean" on her house. 
I was so distracted by the poor writing and weak plot, that I decided to read with a pencil in hand to mark all of the horrible grammar and spelling. 
Please don't waste your money. I too, believe that the good reviews must have been written by the author's relatives. I will not put much faith in the reviews from now on!
"""

In [163]:
print(summarizer_facebook(ARTICLE_review, 
                        #   max_length=70, 
                        #   min_length=1, 
                          do_sample=False))

[{'summary_text': "I was so distracted by the poor writing and weak plot, that I decided to read with a pencil in hand to mark all of the horrible grammar and spelling. I too, believe that the good reviews must have been written by the author's relatives. I will not put much faith in the reviews from now on!"}]


In [143]:
rouge = evaluate.load('rouge')
predictions = ["I was so distracted by the poor writing and weak plot, that I decided to read with a pencil in hand to mark all of the horrible grammar and spelling. I too, believe that the good reviews must have been written by the author's relatives. I will not put much faith in the reviews from now on!"]
references = ["Awful beyond belief!"]
results = rouge.compute(predictions=predictions,
                        references=references,
                        # tokenizer=lambda x: x.split()
                            )
print(results)

{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [144]:
ARTICLE_dialogue = """Jane: Good morning, Doctor Rudra, how are you doing?

Doctor Rudra: Good morning, Jane. I am doing well. And you?

Jane: I’m great, thank you. This is my friend Leila. She is thinking about joining the hospital but she has a few questions about the administration there. Would you mind telling her about the administration, please?

Doctor Rudra: Hello, Leila! It’s a pleasure to meet you. I’m more than happy to speak with you. Please stop by my chamber tomorrow.

Leila: It’s a pleasure to meet you, Doctor. Thank you so much for helping us.

Doctor Rudra: Don’t mention it. Hopefully, I will be able to help you out in this matter.
"""

In [146]:
print(summarizer_facebook(ARTICLE_dialogue, max_length=50, min_length=30, do_sample=False))

[{'summary_text': 'Jane: Good morning, Doctor Rudra, how are you doing? Rudra: I am doing well. Jane: I’m great, thank you. This is my friend Leila. She is thinking about joining the'}]


---

# philschmid/bart-large-cnn-samsum

**Evaluation results**

eval_rouge1	42.621

eval_rouge2	21.9825

eval_rougeL	33.034

eval_rougeLsum	39.6783

test_rouge1	41.3174

test_rouge2	20.8716

test_rougeL	32.1337

test_rougeLsum	38.4149

In [149]:
from transformers import pipeline
summarizer_philschmid= pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

In [150]:
conversation_from_docs = '''Jeff: Can I train a 🤗 Transformers model on Amazon SageMaker? 
Philipp: Sure you can use the new Hugging Face Deep Learning Container. 
Jeff: ok.
Jeff: and how can I get started? 
Jeff: where can I find documentation? 
Philipp: ok, ok you can find everything here. https://huggingface.co/blog/the-partnership-amazon-sagemaker-and-hugging-face                                           
'''

In [151]:
summarizer_philschmid(conversation_from_docs)

[{'summary_text': "Jeff wants to train a Transformers model on Amazon SageMaker. He can use the new Hugging Face Deep Learning Container. Jeff can find the documentation on Huggingface's blog.    .   The blog is available at: https://huggingface.co/blog/the-partnership-amazon-sagemaker-and-hugling-face."}]

In [152]:
conversation_test = '''Waiter: Hello, good evening. Can I start you off with some refreshing drink?

Rana: Yes. I’ll have iced tea, please.

Amal: And I’ll have a chocolate cold coffee.

Waiter: Ok. Should I take your order now, or do you need a few minutes more?

Rana: No no we are ready, you can take the order. I’ll have the corn mushroom soup to start, and the grilled chicken with mashed potatoes and peas. And, please also bring a bowl of garlic rice.

Waiter: Sure sir. How do you want the chicken— low spicy, medium, or high on spice?

Rana: Medium spice, please.

Amal: And I’ll just have the beef, with bread and a salad.
'''

In [153]:
summarizer_philschmid(conversation_test)

[{'summary_text': "Waiter will take Rana's and Amal's order. Rana will have the corn mushroom soup, grilled chicken with mashed potatoes and peas and garlic rice. Amal will have beef, with bread, salad and a beef sandwich. She will also have a chocolate cold coffee."}]

In [156]:
ARTICLE_bbc_news = """Residents of a tiny Swiss village have all been evacuated because of the risk of an imminent rockslide.

Brienz's fewer than 100 villagers were given just 48 hours to pack what they could and abandon their homes.

Even the dairy cows were loaded up for departure after geologists warned a rockfall was imminent.

Two million cubic metres of rock is coming loose from the mountain above, and a rockslide could obliterate the village.

The development has raised questions about the safety of some mountain communities, as global warming changes the alpine environment.

Brienz, in the eastern canton of Graubünden, is now empty.

The village has been judged a geological risk for some time and is built on land that is subsiding down towards the valley, causing the church spire to lean and large cracks to appear in buildings.

As the minutes ticked towards the deadline to leave, even Brienz's dairy cows were being taken to safety.

The residents, some young, some old, families, farmers and professional couples, had two days to abandon their homes.

They were asked earlier this week to evacuate the village by Friday evening.

Switzerland's Alpine regions are especially sensitive to global warming - as the permafrost high in the mountains begins to thaw, the rock becomes more unstable.

This particular mountain has always been unstable, but recently the rock has been shifting faster and faster.

Days of heavy rain could bring two million cubic metres of loosened rock crashing down the mountainside onto the village, scientists warned.

Now the villagers must wait, in temporary accommodation, for the rock to fall - and hope it misses their homes.
"""

In [157]:
summarizer_philschmid(ARTICLE_bbc_news)

[{'summary_text': 'The residents of Brienz, in the eastern canton of Graubünden, Switzerland, were asked to leave their homes by Friday evening due to the risk of a rockslide. Two million cubic metres of loose rock is coming loose from the mountain and could obliterate the village. Even the dairy cows were loaded up for departure.'}]

In [159]:
rouge = evaluate.load('rouge')
predictions = ["The residents of Brienz, in the eastern canton of Graubünden, Switzerland, were asked to leave their homes by Friday evening due to the risk of a rockslide. Two million cubic metres of loose rock is coming loose from the mountain and could obliterate the village. Even the dairy cows were loaded up for departure."]
references = [human_produced_summary_bbc_news]
results = rouge.compute(predictions=predictions,
                        references=references,
                        # tokenizer=lambda x: x.split()
                            )
print(results)

{'rouge1': 0.38983050847457623, 'rouge2': 0.13793103448275862, 'rougeL': 0.22033898305084743, 'rougeLsum': 0.30508474576271183}


In [ ]:
ARTICLE_review= """I feel I have to write to keep others from wasting their money. 
This book seems to have been written by a 7th grader with poor grammatical skills for her age! 
As another reviewer points out, there is a misspelling on the cover, and I believe there is at least one per chapter. 
For example, it was mentioned twice that she had a "lean" on her house. 
I was so distracted by the poor writing and weak plot, that I decided to read with a pencil in hand to mark all of the horrible grammar and spelling. 
Please don't waste your money. I too, believe that the good reviews must have been written by the author's relatives. I will not put much faith in the reviews from now on!
"""

In [165]:
summarizer_philschmid(ARTICLE_review)

[{'summary_text': 'The book seems to have been written by a 7th grader with poor grammatical skills for her age. There is a misspelling on the cover and there are at least one per chapter. I was so distracted by the poor writing and weak plot that I decided to read with a pencil in hand to mark all of the horrible grammar and spelling.'}]

In [166]:
rouge = evaluate.load('rouge')
predictions = ["The book seems to have been written by a 7th grader with poor grammatical skills for her age. There is a misspelling on the cover and there are at least one per chapter. I was so distracted by the poor writing and weak plot that I decided to read with a pencil in hand to mark all of the horrible grammar and spelling."]
references = ["Awful beyond belief!"]
results = rouge.compute(predictions=predictions,
                        references=references,
                        # tokenizer=lambda x: x.split()
                            )
print(results)

{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}
